<center><h2>Jane Street Market Prediction | xgb with treelite  </h2></center><hr>



using [Treelite](https://treelite.readthedocs.io/en/latest/index.html) for a faster inference with a GBDT model. 

![](https://treelite.readthedocs.io/en/latest/_static/benchmark_plot.svg)

Treelite has been used in work when the inference time of a GBDT plays an important role in deployment. **using treelite boosts XGB's inference speed 2-3x**.

Such acceleration may be helpful for, say, model ensembles because the inference time in this competition is quite limited.



# Install treelite

In [ ]:
!pip --quiet install ../input/treelite/treelite-0.93-py3-none-manylinux2010_x86_64.whl

In [ ]:
!pip --quiet install ../input/treelite/treelite_runtime-0.93-py3-none-manylinux2010_x86_64.whl

In [ ]:
import numpy as np
import pandas as pd

import os, sys
import gc
import math
import random
import pathlib
from tqdm import tqdm
from typing import List, NoReturn, Union, Tuple, Optional, Text, Generic, Callable, Dict
from sklearn.preprocessing import MinMaxScaler, StandardScaler, QuantileTransformer
from sklearn.decomposition import PCA
from sklearn import linear_model
import operator
import xgboost as xgb
import lightgbm as lgb
from tqdm import tqdm

# treelite
import treelite
import treelite_runtime 

# visualize
import matplotlib.pyplot as plt
import matplotlib.style as style
import seaborn as sns
from matplotlib_venn import venn2
from matplotlib import pyplot
from matplotlib.ticker import ScalarFormatter
sns.set_context("talk")
style.use('fivethirtyeight')
pd.options.display.max_columns = None

import warnings
warnings.filterwarnings('ignore')

# Config

In [ ]:
SEED = 2021 # Happy new year!
# INPUT_DIR = '../input/jane-street-market-prediction/'
START_DATE = 85
INPUT_DIR = '../input/janestreet-save-as-feather/'
TRADING_THRESHOLD = 0.50 # 0 ~ 1: The smaller, the more aggressive

# Load data

In [ ]:
os.listdir(INPUT_DIR)

['example_test.feather',
 'features.feather',
 '__results__.html',
 'example_sample_submission.feather',
 '__resultx__.html',
 '__notebook__.ipynb',
 '__output__.json',
 'train.feather',
 'custom.css']

In [ ]:
%%time

def load_data(input_dir=INPUT_DIR):
    train = pd.read_feather(pathlib.Path(input_dir + 'train.feather'))
    features = pd.read_feather(pathlib.Path(input_dir + 'features.feather'))
    example_test = pd.read_feather(pathlib.Path(input_dir + 'example_test.feather'))
    ss = pd.read_feather(pathlib.Path(input_dir + 'example_sample_submission.feather'))
    return train, features, example_test, ss

train, features, example_test, ss = load_data(INPUT_DIR)

CPU times: user 532 ms, sys: 3.15 s, total: 3.68 s
Wall time: 8.96 s


In [ ]:
print(train.shape)
train.head()

(2390491, 138)


,date,weight,resp_1,resp_2,resp_3,resp_4,resp,feature_0,feature_1,feature_2,feature_3,feature_4,feature_5,feature_6,feature_7,feature_8,feature_9,feature_10,feature_11,feature_12,feature_13,feature_14,feature_15,feature_16,feature_17,feature_18,feature_19,feature_20,feature_21,feature_22,feature_23,feature_24,feature_25,feature_26,feature_27,feature_28,feature_29,feature_30,feature_31,feature_32,feature_33,feature_34,feature_35,feature_36,feature_37,feature_38,feature_39,feature_40,feature_41,feature_42,feature_43,feature_44,feature_45,feature_46,feature_47,feature_48,feature_49,feature_50,feature_51,feature_52,feature_53,feature_54,feature_55,feature_56,feature_57,feature_58,feature_59,feature_60,feature_61,feature_62,feature_63,feature_64,feature_65,feature_66,feature_67,feature_68,feature_69,feature_70,feature_71,feature_72,feature_73,feature_74,feature_75,feature_76,feature_77,feature_78,feature_79,feature_80,feature_81,feature_82,feature_83,feature_84,feature_85,feature_86,feature_87,feature_88,feature_89,feature_90,feature_91,feature_92,feature_93,feature_94,feature_95,feature_96,feature_97,feature_98,feature_99,feature_100,feature_101,feature_102,feature_103,feature_104,feature_105,feature_106,feature_107,feature_108,feature_109,feature_110,feature_111,feature_112,feature_113,feature_114,feature_115,feature_116,feature_117,feature_118,feature_119,feature_120,feature_121,feature_122,feature_123,feature_124,feature_125,feature_126,feature_127,feature_128,feature_129,ts_id
0,0,0.000000,0.009916,0.014079,0.008773,0.001390,0.006270,1,-1.872746,-2.191242,-0.474163,-0.323046,0.014688,-0.002484,NaN,NaN,-0.989982,-1.055090,NaN,NaN,-2.667671,-2.001475,-1.703595,-2.196892,NaN,NaN,1.483295,1.307466,NaN,NaN,1.175200,0.967805,1.608410,1.319365,NaN,NaN,-0.515073,-0.448988,NaN,NaN,-2.429812,-2.206423,-3.593120,-2.868358,0.112697,0.053157,-0.539956,-0.692187,3.491282,-1.684889,1.337123,-0.328607,1.689207,-1.052243,-1.870885,-1.789342,-1.574173,-1.120820,-0.571920,-1.093033,0.703515,5.936281,NaN,3.315812,1.291338,2.468825,2.490069,-1.148239,-0.961935,-2.263944,-2.158765,-5.012022,-2.006825,-1.284090,-2.141697,-2.054935,-1.851203,-1.431184,-1.634481,NaN,-0.373934,NaN,0.559241,0.891368,0.271700,NaN,-1.521125,NaN,3.045337,3.260512,0.683558,NaN,-0.109194,NaN,0.488806,1.447504,-2.790902,NaN,1.158770,NaN,3.754522,7.137163,-1.863069,NaN,0.434466,NaN,-0.292035,0.317003,-2.605820,NaN,2.896986,NaN,1.485813,4.147254,-2.238831,NaN,-0.892724,NaN,-0.156332,0.622816,-3.921523,NaN,2.561593,NaN,3.457757,6.649580,-1.472686,NaN,NaN,1.168391,8.313583,1.782433,14.018213,2.653056,12.600292,2.301488,11.445807,0
1,0,16.673515,-0.002828,-0.003226,-0.007319,-0.011114,-0.009792,-1,-1.349537,-1.704709,0.068058,0.028432,0.193794,0.138212,NaN,NaN,-0.151877,-0.384952,NaN,NaN,1.225838,0.789076,1.110580,1.102281,NaN,NaN,-0.590600,-0.625682,NaN,NaN,-0.543425,-0.547486,-0.706600,-0.667806,NaN,NaN,0.910558,0.914465,NaN,NaN,2.137454,2.080459,2.819291,2.483965,-0.086755,-0.082687,0.368431,0.469196,5.711996,-2.215132,0.796703,-1.140081,0.716617,-0.059431,-0.198920,-0.326697,-0.381770,1.435607,3.401393,2.486748,-2.014598,-0.390588,NaN,-0.027262,-1.886927,-1.706450,-0.888236,-1.138294,-0.954461,-1.350633,-1.459546,-4.564815,-2.651966,-1.620014,-2.240625,-2.147273,-0.255224,3.202946,-0.535872,NaN,-0.050948,NaN,0.141089,0.058363,0.131190,NaN,-0.121239,NaN,0.677553,0.045842,-0.124616,NaN,-0.007004,NaN,-0.410491,-0.024323,-3.012654,NaN,1.157671,NaN,1.297679,1.281956,-2.427595,NaN,0.024913,NaN,-0.413607,-0.073672,-2.434546,NaN,0.949879,NaN,0.724655,1.622137,-2.209020,NaN,-1.332492,NaN,-0.586619,-1.040491,-3.946097,NaN,0.983440,NaN,1.357907,1.612348,-1.664544,NaN,NaN,-1.178850,1.777472,-0.915458,2.831612,-1.417010,2.297459,-1.304614,1.898684,1
2,0,0.000000,0.025134,0.027607,0.033406,0.034380,0.023970,-1,0.812780,-0.256156,0.806463,0.400221,-0.614188,-0.354800,NaN,NaN,5.448261,2.668029,NaN,NaN,3.836342,2.183258,3.902698,3.045431,NaN,NaN,-1.141082,-0.979962,NaN,NaN,-1.157585,-0.966803,

In [ ]:
del features, example_test, ss
gc.collect()

20

In [ ]:
# reduce train
train = train.query(f'date > {START_DATE}')

# Model fitting
For now, let's use a simple XGBoost which is also used in the example in the Numerai Tournament.

In [ ]:
# remove weight = 0 for saving memory 
original_size = train.shape[0]
train = train.query('weight > 0').reset_index(drop=True)

print('Train size reduced from {:,} to {:,}.'.format(original_size, train.shape[0]))

Train size reduced from 1,862,597 to 1,571,415.


In [ ]:
# feats
feats = train.columns[train.columns.str.startswith('feature')].values.tolist()

print('{} features used'.format(len(feats)))

130 features used


In [ ]:
# target
train['action'] = train['resp'] * train['weight']


In [ ]:
%%time

# same hyperparameters from https://www.kaggle.com/hamditarek/market-prediction-xgboost-with-gpu-fit-in-1min?scriptVersionId=48127254
params = {
    'colsample_bytree': 0.72,                 
    'learning_rate': 0.08,
    'max_depth': 7,
    'subsample': 0.8,
    'seed': SEED,
    'n_estimators': 480,
#     'tree_method': 'gpu_hist' # Let's use GPU for a faster experiment
}
params["objective"] = 'binary:logistic'
params["eval_metric"] = 'logloss'
train['action'] = 1 * (train['action'] > 0) # binary classification
# model = xgb.XGBClassifier(**params)
# model.fit(train[feats], train['action'], verbose=100)

CPU times: user 6.47 ms, sys: 1.07 ms, total: 7.54 ms
Wall time: 6.98 ms


In [ ]:
# fit
dtrain = xgb.DMatrix(train[feats].values, label=train['action'].values)
bst = xgb.train(params, dtrain, 100, [(dtrain, 'train')])

[02:58:23] WARNING: ../src/learner.cc:516: 
Parameters: { n_estimators } might not be used.

  This may not be accurate due to some parameters are only used in language bindings but
  passed down to XGBoost core.  Or some parameters are not used but slip through this
  verification. Please open an issue if you find above cases.


[0]	train-logloss:0.69078
[1]	train-logloss:0.69115
[2]	train-logloss:0.69129
[3]	train-logloss:0.69095
[4]	train-logloss:0.69080
[5]	train-logloss:0.69068
[6]	train-logloss:0.69035
[7]	train-logloss:0.69001
[8]	train-logloss:0.68977
[9]	train-logloss:0.68944
[10]	train-logloss:0.68923
[11]	train-logloss:0.68891
[12]	train-logloss:0.68868
[13]	train-logloss:0.68849
[14]	train-logloss:0.68819
[15]	train-logloss:0.68802
[16]	train-logloss:0.68781
[17]	train-logloss:0.68762
[18]	train-logloss:0.68742
[19]	train-logloss:0.68723
[20]	train-logloss:0.68708
[21]	train-logloss:0.68687
[22]	train-logloss:0.68674
[23]	train-logloss:0.68656
[24]	train-logloss:0.68639
[25

# Compile with Treelite
Simply follow the tutorial: https://treelite.readthedocs.io/en/latest/tutorials/first.html

In [ ]:
# pass to treelite
model = treelite.Model.from_xgboost(bst)

In [ ]:
# generate shared library
toolchain = 'gcc'
model.export_lib(toolchain=toolchain, libpath='./mymodel.so',
                 params={'parallel_comp': 32}, verbose=True)

[03:33:00] /workspace/src/compiler/ast_native.cc:44: Using ASTNativeCompiler
[03:33:00] /workspace/src/compiler/ast/split.cc:29: Parallel compilation enabled; member trees will be divided into 32 translation units.
[03:33:00] /workspace/src/c_api/c_api.cc:286: Code generation finished. Writing code to files...
[03:33:00] /workspace/src/c_api/c_api.cc:291: Writing file recipe.json...
[03:33:00] /workspace/src/c_api/c_api.cc:291: Writing file tu22.c...
[03:33:00] /workspace/src/c_api/c_api.cc:291: Writing file tu24.c...
[03:33:00] /workspace/src/c_api/c_api.cc:291: Writing file tu21.c...
[03:33:00] /workspace/src/c_api/c_api.cc:291: Writing file tu20.c...
[03:33:00] /workspace/src/c_api/c_api.cc:291: Writing file tu6.c...
[03:33:00] /workspace/src/c_api/c_api.cc:291: Writing file tu5.c...
[03:33:00] /workspace/src/c_api/c_api.cc:291: Writing file tu8.c...
[03:33:00] /workspace/src/c_api/c_api.cc:291: Writing file tu3.c...
[03:33:00] /workspace/src/c_api/c_api.cc:291: Writing file tu2.c..

In [ ]:
# predictor from treelite
predictor = treelite_runtime.Predictor('./mymodel.so', verbose=True)

[03:33:06] /opt/conda/lib/python3.7/site-packages/treelite_runtime/predictor.py:309: Dynamic shared library /kaggle/working/mymodel.so has been successfully loaded into memory


# Speed Test
I use a dummy data to see how faster the inference with treelite can get.

In [ ]:
# dummy data
np.random.seed(SEED)
N = 10000
dummy_data = np.random.rand(N, len(feats))

In [ ]:
%%time

# normal xgb
predicted_normal = bst.predict(xgb.DMatrix(dummy_data))

CPU times: user 135 ms, sys: 4.99 ms, total: 140 ms
Wall time: 75.4 ms


In [ ]:
%%time

# treelite
batch = treelite_runtime.Batch.from_npy2d(dummy_data)
predicted_treelite = predictor.predict(batch)

CPU times: user 52.9 ms, sys: 3 ms, total: 55.9 ms
Wall time: 14.5 ms


In [ ]:
predicted_normal == predicted_treelite

array([ True,  True,  True, ...,  True,  True,  True])

# Submit

In [ ]:
import janestreet
env = janestreet.make_env() # initialize the environment
iter_test = env.iter_test() # an iterator which loops over the test set

In [ ]:
for (test_df, pred_df) in tqdm(iter_test):
    if test_df['weight'].item() > 0:
        # inference with treelite
        batch = treelite_runtime.Batch.from_npy2d(test_df[feats].values)
        pred_df.action = (predictor.predict(batch) > TRADING_THRESHOLD).astype('int')
    else:
        pred_df.action = 0
    env.predict(pred_df)

15219it [02:49, 90.00it/s]
